In [1]:
import pandas as pd
import requests
from sklearn import preprocessing 


In [2]:
start = 0
end = 50

In [3]:
pd.set_option('display.max_rows', 500)


In [4]:
match_summary_df = pd.read_csv('../data/encoded_match_data.csv')

In [5]:
match_summary_df = match_summary_df[match_summary_df['league'] == 0][start:end]


In [6]:
len(match_summary_df)

50

In [7]:
match_summary_df.columns

Index(['league', 'format', 'game_in_format', 'red_team_outcome',
       'blue_team_outcome', 'season_format', 'riot_match_id', 'min_start_time',
       'month', 'day', 'year', 'red_team_id', 'blue_team_id', 'winner_id',
       'loser_id'],
      dtype='object')

In [8]:
match_summary_df.head(10)

,league,format,game_in_format,red_team_outcome,blue_team_outcome,season_format,riot_match_id,min_start_time,month,day,year,red_team_id,blue_team_id,winner_id,loser_id
0,0,2,1,0,1,0,111561337007371296,2024-04-14 06:00:00+00:00,4,14,2024,2144,2145,2145,2144
1,0,2,2,0,1,0,111561337007371297,2024-04-14 06:00:00+00:00,4,14,2024,2145,2144,2144,2145
2,0,2,3,1,0,0,111561337007371298,2024-04-14 07:15:17+00:00,4,14,2024,2144,2145,2144,2145
3,0,2,4,0,1,0,111561337007371299,2024-04-14 07:53:09+00:00,4,14,2024,2144,2145,2145,2144
4,0,2,5,1,0,0,111561337007371300,2024-04-14 08:23:02+00:00,4,14,2024,2145,2144,2145,2144
5,0,2,1,0,1,0,111561337007371290,2024-04-13 06:00:00+00:00,4,13,2024,2144,2122,2122,2144
6,0,2,2,0,1,0,111561337007371291,2024-04-13 06:00:00+00:00,4,13,2024,2122,2144,2144,2122
7,0,2,3,0,1,0,111561337007371292,2024-04-13 07:15:17+00:00,4,13,2024,2122,2144,2144,2122
8,0,2,4,1,0,0,111561337007371293,2024-04-13 07:50:26+00:00,4,13,2024,2144,2122,2144,2122
9,0,2,1,0,1,0,111561337007371284,2024-04-07 06:00:00+00:00,4,7,2024,2118,2144,2144,2118


In [9]:
columns=match_summary_df.columns.to_list()
columns.extend(['patch_version'])

for clr in ['blue','red']:
    columns.append(F"{clr}_team_id")
    for pos in ['top','jungle','mid','bot','supp']:
        col = F"{clr}_{pos}_player"
        columns.append(col)
        col = F"{clr}_{pos}_champ"
        columns.append(col)

columns.extend(['rfc460Timestamp'])
for clr in ['blue','red']:
    for info in ['total_gold','inhibitors','towers','barons','total_kills','ocean_drakes','cloud_drakes','mountain_drakes','chemtech_drakes','infernal_drakes','hextech_drakes','elder_drakes']:
        col = F"{clr}_team_{info}"
        columns.append(col)

    for pos in ['top','jungle','mid','bot','supp']:
        for info in ['total_gold','level','kills','deaths','assists','creep_score','current_health','max_health']:
            col = F"{clr}_{pos}_{info}"
            columns.append(col)




In [10]:
columns

['league',
 'format',
 'game_in_format',
 'red_team_outcome',
 'blue_team_outcome',
 'season_format',
 'riot_match_id',
 'min_start_time',
 'month',
 'day',
 'year',
 'red_team_id',
 'blue_team_id',
 'winner_id',
 'loser_id',
 'patch_version',
 'blue_team_id',
 'blue_top_player',
 'blue_top_champ',
 'blue_jungle_player',
 'blue_jungle_champ',
 'blue_mid_player',
 'blue_mid_champ',
 'blue_bot_player',
 'blue_bot_champ',
 'blue_supp_player',
 'blue_supp_champ',
 'red_team_id',
 'red_top_player',
 'red_top_champ',
 'red_jungle_player',
 'red_jungle_champ',
 'red_mid_player',
 'red_mid_champ',
 'red_bot_player',
 'red_bot_champ',
 'red_supp_player',
 'red_supp_champ',
 'rfc460Timestamp',
 'blue_team_total_gold',
 'blue_team_inhibitors',
 'blue_team_towers',
 'blue_team_barons',
 'blue_team_total_kills',
 'blue_team_ocean_drakes',
 'blue_team_cloud_drakes',
 'blue_team_mountain_drakes',
 'blue_team_chemtech_drakes',
 'blue_team_infernal_drakes',
 'blue_team_hextech_drakes',
 'blue_team_elde

In [11]:
match_data_df = pd.DataFrame(columns=columns)

In [12]:
match_data_df

,league,format,game_in_format,red_team_outcome,blue_team_outcome,season_format,riot_match_id,min_start_time,month,day,...,red_bot_current_health,red_bot_max_health,red_supp_total_gold,red_supp_level,red_supp_kills,red_supp_deaths,red_supp_assists,red_supp_creep_score,red_supp_current_health,red_supp_max_health


In [13]:
def extract_fixed_info(json_data):
    ret_data = dict()
    ret_data['patch_version'] = json_data['gameMetadata']['patchVersion']
    ret_data['blue_team_id'] = json_data['gameMetadata']['blueTeamMetadata']['esportsTeamId']
    for i,pos in enumerate(['top','jungle','mid','bot','supp']):

        ret_data[F'blue_{pos}_player'] = json_data['gameMetadata']['blueTeamMetadata']['participantMetadata'][i]['esportsPlayerId']
        ret_data[F'blue_{pos}_champ'] = json_data['gameMetadata']['blueTeamMetadata']['participantMetadata'][i]['championId']

    ret_data['red_team_id'] = json_data['gameMetadata']['redTeamMetadata']['esportsTeamId']
    for i,pos in enumerate(['top','jungle','mid','bot','supp']):

        ret_data[F'red_{pos}_player'] = json_data['gameMetadata']['redTeamMetadata']['participantMetadata'][i]['esportsPlayerId']
        ret_data[F'red_{pos}_champ'] = json_data['gameMetadata']['redTeamMetadata']['participantMetadata'][i]['championId']

    return ret_data

In [14]:
def encode_dragons(team,info):
    ret_data = dict()
    for drake in ['ocean_drakes','cloud_drakes','mountain_drakes','chemtech_drakes','infernal_drakes','hextech_drakes','elder_drakes']:
        ret_data[F"{team}_{drake}"] = 0
    for d in info:
        k = F"{team}_{d}_drakes"
        ret_data[k] = ret_data[k] + 1
    
    return ret_data

In [15]:
def extract_changing_info(json_data):
    ret_data = dict()
    # ret_data['rfc460Timestamp'] = json_data['rfc460Timestamp']
    for clr in ['blue','red']:
        ret_data[F'{clr}_team_total_gold'] = json_data[F'{clr}Team']['totalGold']
        ret_data[F'{clr}_team_inhibitors'] = json_data[F'{clr}Team']['inhibitors']
        ret_data[F'{clr}_team_towers'] = json_data[F'{clr}Team']['towers']
        ret_data[F'{clr}_team_barons'] = json_data[F'{clr}Team']['barons']
        ret_data[F'{clr}_team_total_kills'] = json_data[F'{clr}Team']['totalKills']
        ret_data.update(encode_dragons(F'{clr}_team',json_data[F'{clr}Team']['dragons']))

        for i,pos in enumerate(['top','jungle','mid','bot','supp']):
            ret_data[F'{clr}_{pos}_total_gold'] = json_data[F'{clr}Team']['participants'][i]['totalGold']
            ret_data[F'{clr}_{pos}_level'] = json_data[F'{clr}Team']['participants'][i]['level']
            ret_data[F'{clr}_{pos}_kills'] = json_data[F'{clr}Team']['participants'][i]['kills']
            ret_data[F'{clr}_{pos}_deaths'] = json_data[F'{clr}Team']['participants'][i]['deaths']
            ret_data[F'{clr}_{pos}_assists'] = json_data[F'{clr}Team']['participants'][i]['assists']
            ret_data[F'{clr}_{pos}_creep_score'] = json_data[F'{clr}Team']['participants'][i]['creepScore']
            ret_data[F'{clr}_{pos}_current_health'] = json_data[F'{clr}Team']['participants'][i]['currentHealth']
            ret_data[F'{clr}_{pos}_max_health'] = json_data[F'{clr}Team']['participants'][i]['maxHealth']


    return ret_data


In [16]:
BASE_URL = 'https://feed.lolesports.com/livestats/v1/window/{}?startingTime={}'

In [17]:
def format_time(t,time_delta):
    conv_time = pd.to_datetime(t) + pd.to_timedelta(time_delta, unit='s')
    conv_time = conv_time.round('10s')
    return "{}T{}.000Z".format(str(conv_time.date()),str(conv_time.time()))

In [18]:
time_delta = 60
temp_list = []

for index, row in match_summary_df.iterrows():
    print(index)
    print(row)
    game_state = "in_game"
    fixed_info = None
    time_elapsed = 0

    while game_state == "in_game":

        url = BASE_URL.format(row['riot_match_id'],format_time(row['min_start_time'],time_elapsed))
        raw_data = requests.get(url)

        time_elapsed = time_elapsed + time_delta
        if raw_data.status_code == 204: # No content returned
            continue

        data = raw_data.json()
        if fixed_info == None:
            fixed_info = extract_fixed_info(data)

        prev_frame = data['frames'][0]
        for frame in data['frames'][1:]:
            delta_info = extract_changing_info(frame)
            if delta_info == prev_frame:
                continue
            delta_info.update(fixed_info)
            delta_info['rfc460Timestamp'] = frame['rfc460Timestamp']
            delta_info.update(row)
            temp_list.append(delta_info)
            prev_frame = delta_info


        game_state = data['frames'][-1]['gameState']


0
league                                       0
format                                       2
game_in_format                               1
red_team_outcome                             0
blue_team_outcome                            1
season_format                                0
riot_match_id               111561337007371296
min_start_time       2024-04-14 06:00:00+00:00
month                                        4
day                                         14
year                                      2024
red_team_id                               2144
blue_team_id                              2145
winner_id                                 2145
loser_id                                  2144
Name: 0, dtype: object
1
league                                       0
format                                       2
game_in_format                               2
red_team_outcome                             0
blue_team_outcome                            1
season_format                    

In [19]:
match_data_df = pd.DataFrame.from_dict(temp_list)

In [20]:
match_data_df.head()

,blue_team_total_gold,blue_team_inhibitors,blue_team_towers,blue_team_barons,blue_team_total_kills,blue_team_ocean_drakes,blue_team_cloud_drakes,blue_team_mountain_drakes,blue_team_chemtech_drakes,blue_team_infernal_drakes,...,red_team_outcome,blue_team_outcome,season_format,riot_match_id,min_start_time,month,day,year,winner_id,loser_id
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,111561337007371296,2024-04-14 06:00:00+00:00,4,14,2024,2145,2144
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,111561337007371296,2024-04-14 06:00:00+00:00,4,14,2024,2145,2144
2,2500,0,0,0,0,0,0,0,0,0,...,0,1,0,111561337007371296,2024-04-14 06:00:00+00:00,4,14,2024,2145,2144
3,2500,0,0,0,0,0,0,0,0,0,...,0,1,0,111561337007371296,2024-04-14 06:00:00+00:00,4,14,2024,2145,2144
4,2500,0,0,0,0,0,0,0,0,0,...,0,1,0,111561337007371296,2024-04-14 06:00:00+00:00,4,14,2024,2145,2144


In [21]:
match_data_df.to_csv(F'../data/match_time_data_LCK_{start}_{end}.csv',index=False)